# CoDeepNEAT demo
## CS081 project checkpoint demo
### Harsha Uppli, Alan Zhao, Gabriel Meyer-Lee

The following notebook demonstrates using CoDeepNEAT to solve the Penn Tree Bank

In [7]:
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras import preprocessing
from keras import backend as K
from math import pi, floor
from random import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras
import collections

In [8]:
max_words = 10000
print('Loading data...')
(x_train_all, y_train_all), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)
print(len(x_train_all), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train_all) + 1
print(num_classes, 'classes')

print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train_all = tokenizer.sequences_to_matrix(x_train_all, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
x_train_all = preprocessing.sequence.pad_sequences(x_train_all, maxlen=30)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=30)[:2240]

# print('x_train shape:', x_train.shape)
# print('x_test shape:', x_test.shape)


print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train_all = keras.utils.to_categorical(y_train_all, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)[:2240]
#Reshaping the input data
# x_train = np.reshape(x_train, (8982, 10000, 1))
# x_test = np.reshape(x_test, (2246, 10000, 1))
# print('y_train shape:', y_train_all.shape)
# print('y_test shape:', y_test.shape)

index = np.array(range(len(x_train_all)))
np.random.shuffle(index)
index_train = index[:7862]
index_val = index[7862:]
x_train = x_train_all[index_train][:7840]
y_train = y_train_all[index_train][:7840]
x_val = x_train_all[index_val]
y_val = y_train_all[index_val]
print('x_train shape', x_train.shape)
print('x_val shape', x_val.shape)
print('x_test shape', x_test.shape)

data = [x_train, y_train, x_val, y_val, x_test, y_test]


Loading data...
8982 train sequences
2246 test sequences
46 classes
Vectorizing sequence data...
Convert class vector to binary class matrix (for use with categorical_crossentropy)
x_train shape (7840, 30)
x_val shape (1120, 30)
x_test shape (2240, 30)


In [9]:
%%file configReuters
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 30
output_nodes        = 46
conv                = False
LSTM                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.0
prob_addLSTM          = 0.9
prob_addlayer         = 0.1
prob_mutatelayer      = 0.4
prob_addmodule        = 0.05
prob_switchmodule     = 0.3
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coefficient      = 5.0
disjoint_coefficient    = 3.0
connection_coefficient  = 0.4
size_coefficient        = 0.8

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configReuters


In [10]:
def fitness(network, data):
    K.set_learning_phase(1)
    network.fit(data[0], data[1],  epochs=5, batch_size = 32)
    K.set_learning_phase(0)
    loss, acc = network.evaluate(data[2], data[3], batch_size=32)
    return acc

In [ ]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configReuters')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(15, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(10, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 2, fitness, data, save_best=True, name='reuters', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="reutersmod_")
    visualize.plot_stats(blueprint_pop.stats, name="reutersbp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [ ]:
evolve(25, True)

-----Generation 0--------
Network 0
Epoch 1/5
7840/7840 [==============================] - 25s - loss: 2.4623 - acc: 0.3417    
Epoch 2/5
7840/7840 [==============================] - 25s - loss: 2.4330 - acc: 0.3494    
Epoch 3/5
7840/7840 [==============================] - 24s - loss: 2.4289 - acc: 0.3494    
Epoch 4/5
7840/7840 [==============================] - 24s - loss: 2.4270 - acc: 0.3494    
Epoch 5/5
1056/1120 [===========================>..] - ETA: 0s
Network 0 Fitness: 0.366964285714
Network 1
Epoch 1/5
7840/7840 [==============================] - 25s - loss: 2.5196 - acc: 0.3401    
Epoch 2/5
7840/7840 [==============================] - 25s - loss: 2.4278 - acc: 0.3494    
Epoch 3/5
7840/7840 [==============================] - 24s - loss: 2.4258 - acc: 0.3494    
Epoch 4/5
7840/7840 [==============================] - 24s - loss: 2.4239 - acc: 0.3494    
Epoch 5/5
1056/1120 [===========================>..] - ETA: 0s
Network 1 Fitness: 0.366964285714
-----Modules-----------


RuntimeError: Can't decrement id ref count (file write failed: time = Sun Dec  3 21:27:43 2017
, filename = 'reuters_best_model_1', file descriptor = 72, errno = 122, error message = 'Disk quota exceeded', buf = 0x12dc1338, total write size = 8192, bytes this sub-write = 8192, bytes actually written = 18446744073709551615, offset = 19262616)